In [1]:
import simpy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pandas as pd
import time
from scipy import stats
import heapq


Queuing theory tells us that for FIFO scheduling the average waiting times are shorter for an
$M/M/n$ queue and a system load $\rho$ and processor capacity $\mu$ than for a single $M/M/1$ queue
with the same load characteristics (and thus an n-fold lower arrival rate). Of course, $\rho$ must be
less than one, but the experiment only becomes interesting when $\rho$ is not much less than one.

$\lambda$ then becomes $\lambda = \rho*n*\mu$.

Code sources: "https://simpy.readthedocs.io/en/latest/examples/bank_renege.html" and "https://www.youtube.com/watch?v=eSNfC-HOl44"

In [60]:
# M/M/1 (M/M/n) queueing system
def generate_interarrival(mu,n,rho):
    return random.expovariate(mu*n*rho)


def source(env, n, rho, mu, servers):
    """Source generates customers randomly based on exponential distribution"""
    i = 0
    while True:
        i += 1
        c = customer(env, i, servers, mu)
        env.process(c)
        t = random.expovariate(mu*n*rho)
        yield env.timeout(t)   

total_waiting_time = []

def customer(env, customer, servers, mu):
    """This generator function models the behavior of a customer in the queue."""
    time_of_arrival = env.now
    print(len(servers.queue))
    with servers.request() as req:    
        #print(env.now, 'customer {} arrives'.format(customer))
        yield req
        time_of_handling = random.expovariate(mu)
        #print(env.now, 'customer {} is being served'.format(customer))
        yield env.timeout(time_of_handling)
        waiting_time = env.now
        #print(env.now, 'customer {} departs'.format(customer))
        total_waiting_time.append(waiting_time-time_of_arrival)

### Setup and start the simulation

# Initial values            
mu = 1  # Parameter mu
rho = 0.9 #[0.7,0.85,0.9,0.95,0.99] # Parameter rho (varying for values close to 1)
n = 2 # Number of servers, in our case 1, 2 and 4

env = simpy.Environment()

# Start processes and run
servers = simpy.Resource(env, capacity=n)
env.process(source(env, n, rho, mu, servers))
env.run(until=200)
print(f"Total customers served: {len(total_waiting_time)}")
print(f"Average waiting time: {sum(total_waiting_time) / len(total_waiting_time) if total_waiting_time else 0}")

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
1
2
0
0
1
2
3
4
3
4
3
4
4
5
6
6
3
4
4
3
3
3
3
4
2
3
4
0
0
0
0
1
2
3
1
2
3
3
4
1
2
2
0
0
0
0
0
1
2
3
3
4
4
5
6
7
5
6
4
1
2
2
3
4
0
0
0
1
1
2
2
3
3
3
3
4
5
6
5
3
4
5
3
1
2
2
3
3
4
4
5
6
7
8
8
8
9
6
6
6
6
7
8
9
6
6
6
4
3
4
3
3
1
1
1
2
3
4
5
6
7
6
7
6
7
7
7
8
8
8
8
7
8
9
5
6
7
6
7
8
9
9
8
8
8
9
8
9
5
5
6
7
8
5
5
6
3
2
2
3
3
4
3
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
1
2
2
3
4
5
5
6
6
7
5
6
7
8
9
9
7
6
7
7
8
8
7
7
7
8
6
6
6
7
8
8
9
9
8
6
7
6
5
6
7
8
9
8
9
10
10
11
10
10
7
8
8
5
6
6
7
8
9
10
11
11
12
13
14
13
11
12
12
12
9
10
10
10
11
9
10
10
10
9
10
9
10
11
12
10
11
12
11
10
10
11
7
7
5
2
3
4
5
6
7
8
7
7
7
7
6
6
3
4
4
5
6
7
6
7
7
7
7
7
8
6
7
6
7
8
9
10
11
11
8
8
9
10
9
6
6
Total customers served: 367
Average waiting time: 3.70169232890415


In [6]:
# Defining function for simulating M/M/n queue system

# Setup and start the simulation
def sim_run(n,rho,mu,sim_duration):
    env = simpy.Environment()
    # Start processes and run
    servers = simpy.Resource(env, capacity=n)
    env.process(source(env, n, rho, mu, servers))
    env.run(until=sim_duration)
    
# Looping over the simulation and extracting mean and standard deviation
# Initial values
sim_duration = 500     
mu = 1  # Parameter mu
rho_vals = [0.9,0.95] #[0.7,0.85,0.92,0.95,0.99]
n = [1] #[1,2,4]

# Running simulation to calculate mean
sim_runs = 20
data = []

for n_servers in n:
    for rho in rho_vals:
        for i in range(sim_runs):
            total_waiting_time = []
            sim_run(n=n_servers, rho=rho, mu=mu, sim_duration=sim_duration)
            averages = np.concatenate((np.arange(100, 1000, 100), np.arange(1000, 10000, 1000), np.arange(10000, 100001, 10000)))
            df = pd.DataFrame({
                'run': [i] * len(averages),
                'rho': [rho] * len(averages),
                'number of servers': [n_servers] * len(averages),
                'number of customers': averages,
                'average waiting time': [np.mean(total_waiting_time[:n_customers]) for n_customers in averages]
            })
            data.append(df)

        print(f'rho={rho} for server number {n_servers} finished')

# Concatenate the list of DataFrames
result_df = pd.concat(data, ignore_index=True)    

rho=0.9 for server number 1 finished
rho=0.95 for server number 1 finished


In [ ]:
# Plot the mean and standard deviation of the waiting time

SJF implementation

#1 Preemptive SJF

In [13]:
def source(env, mu, n, rho, queue):
    i = 0
    while True:
        i +=1
        time_of_handling = random.expovariate(mu)
        arrival_time = env.now
        heapq.heappush(queue, (time_of_handling, arrival_time))
        #print(f"Bisy: {servers.count}")
        #print(queue)
        yield env.timeout(random.expovariate(mu*n*rho))

def do_work(env, client, servers, time_to_finish, total_waiting_time):
    with servers.request() as req:
        yield req
        #print(f"Number of servers in util: {servers.count}")
        yield env.timeout(time_to_finish)
        total_waiting_time.append(env.now - client)

def customer_initialise(env, servers, queue, total_waiting_time):
    while True:
        if queue and servers.count < servers.capacity:
            print(len(queue))
            time_to_finish, client = heapq.heappop(queue)
            env.process(do_work(env, client, servers, time_to_finish, total_waiting_time))
        else:
            yield env.timeout(0.0001)

mu = 1.0          
n = 1             
rho = 1       
sim_time = 200    

env = simpy.Environment()
servers = simpy.Resource(env,capacity=n)
priority_queue = []
total_waiting_time = []
print(priority_queue)
env.process(source(env, mu, n, rho, priority_queue))
env.process(customer_initialise(env, servers, priority_queue, total_waiting_time))
env.run(until=sim_time)

print(f"Total customers served: {len(total_waiting_time)}")
print(f"Average waiting time: {sum(total_waiting_time) / len(total_waiting_time)}")

[]
1
1
1
1
1
1
1
1
1
1
1
4
3
2
1
3
2
1
1
5
4
3
2
1
7
6
5
4
3
2
1
3
2
1
2
1
1
1
4
3
2
1
6
5
4
3
2
1
5
4
3
2
1
2
1
1
1
3
2
1
2
1
1
1
1
1
3
2
1
5
4
3
2
1
4
3
2
1
1
1
1
1
1
1
1
2
1
4
3
2
1
5
4
3
2
1
10
9
8
7
6
5
4
3
2
1
4
3
2
1
6
5
4
3
2
1
7
6
5
4
3
2
1
7
6
5
4
3
2
1
2
1
1
1
1
1
1
1
1
3
2
1
2
1
1
1
2
1
4
3
2
1
5
4
3
2
1
8
7
6
5
4
3
2
1
9
8
7
6
5
4
3
2
1
8
7
6
5
4
3
2
1
5
4
3
2
1
2
1
4
3
2
1
6
5
4
3
2
1
Total customers served: 197
Average waiting time: 3.5737293113643296


#2 Non preemptive SJF (halting the precesses if faster customer arrives)

In [16]:

class Client:
    def __init__(self, time_of_arrival, time_of_handling):
        self.time_of_arrival = time_of_arrival
        self.time_of_handling = time_of_handling

def source_preemptive(env, mu, n, rho, queue):
    while True:
        time_of_handling = random.expovariate(mu)
        arrival_time = env.now
        heapq.heappush(queue, (time_of_handling, Client(arrival_time, time_of_handling)))
        yield env.timeout(random.expovariate(mu*n*rho))

def do_work_preemptive(env, client, servers, time_to_finish, total_waiting_time, queue):
    with servers.request() as req:
        yield req
        if time_to_finish <= 0.01:
            yield env.timeout(time_to_finish)
            total_waiting_time.append(env.now - client.time_of_arrival)
        else:
            yield env.timeout(0.01)
            arrival_time = client.time_of_arrival
            time_of_handling = client.time_of_handling - 0.01
            heapq.heappush(queue, (time_of_handling, Client(arrival_time, time_of_handling)))
            
def customer_initialise_preemptive(env, servers, queue, total_waiting_time, q_arr):
    while True:
        if queue and servers.count < servers.capacity:
            time_to_finish, client = heapq.heappop(queue)
            env.process(do_work_preemptive(env, client, servers, time_to_finish, total_waiting_time, queue))
        else:
            yield env.timeout(0.0001)

mu = 1.0         
n = 2             
rho = 0.95         
sim_time = 200

env = simpy.Environment()
servers = simpy.Resource(env,capacity=n)
priority_queue = []
total_waiting_time = []
q_arr = [] 
env.process(source_preemptive(env, mu, n, rho, priority_queue))
env.process(customer_initialise_preemptive(env, servers, priority_queue, total_waiting_time, q_arr))
env.run(until=sim_time)
print(f"Que length mean: {q_arr}")
print(f"Total customers served: {len(total_waiting_time)}")
print(f"Average waiting time: {sum(total_waiting_time) / len(total_waiting_time)}")

Que length mean: []
Total customers served: 385
Average waiting time: 3.5227148826796606


#3 Data analysis

In [46]:
def sim_run(n,rho,mu,sim_time):
    env = simpy.Environment()
    servers = simpy.Resource(env,capacity=n)
    priority_queue = []
    total_waiting_time = []
    env.process(source_preemptive(env, mu, n, rho, priority_queue))
    env.process(customer_initialise_preemptive(env, servers, priority_queue, total_waiting_time))
    env.run(until=sim_time)
    return len(total_waiting_time), sum(total_waiting_time) / len(total_waiting_time), 
rho_vals = [0.9,0.95]
serv = [1, 2, 3, 4]
sim_time = 5000
mu = 1.0         
simulation = ([[[sim_run(n,rho,mu,sim_time) for _ in range(100)] for rho in rho_vals] for n in serv])





KeyboardInterrupt: 

In [ ]:
fstd = lambda array: [np.std(sub_array) for sub_array in array]
fmean = lambda array: [np.mean(sub_array) for sub_array in array]
fn = lambda array: [len(sub_array) for sub_array in array]
std_v = list(map(fstd, res))
mean_v = list(map(fmean, res))
n = list(map(fn, res))
